In [90]:
%matplotlib notebook
#%run script
import os

os.sys.path.append("..") #script path

import defs
import nibabel as nib
import numpy as np
from scipy import stats
import pandas as pd

In [91]:
KEY_ID = 9
seg_prob_dir = "../../fine_stage/hr3d_h_e-3_16-72_d_100k__full_e-4_24-24_gdsc_1024s_dil13/output/50000_test_prob/"
seg_ref_dir = "../../data/HaN_MICCAI2015_Dataset/nii_segmentations_orig/"

In [99]:
def addToDict(dictionary, key, value):
    if dictionary.get(key) is None:
        dictionary[key] = [value]
    else:
        dictionary[key].append(value)
        
prob_dict = dict()
df = pd.DataFrame(columns=['file','organ','vox_cnt','mean','std', 'median','max', 'min'])

for prob_file in os.listdir(seg_prob_dir):
    for ref_file in os.listdir(seg_ref_dir):
        file_id = prob_file[:KEY_ID]
        if file_id in ref_file[:KEY_ID]:   
            
            prob_np = nib.load(os.path.join(seg_prob_dir, prob_file)).get_fdata()
            ref_np = nib.load(os.path.join(seg_ref_dir, ref_file)).get_fdata().astype(np.uint8)
            print("evaluating: ", file_id, prob_np.shape, ref_np.shape)
            for key, value in defs.LABELS.items():
                label_prob = prob_np[...,value].squeeze()[ref_np == value]
                cnt, minmax, mean, std, _, _ = stats.describe(label_prob)
                median = np.median(label_prob)
                df.append({'file':file_id, 'organ':key, 'vox_cnt':cnt, 'mean':mean, 'std':std, 'median':median,
                          'max':minmax[1], 'min':minmax[0]}, ignore_index=True)
                
                addToDict(prob_dict, key, label_prob)

evaluating:  0522c0669 (512, 512, 137, 1, 8) (512, 512, 137)
evaluating:  0522c0555 (512, 512, 263, 1, 8) (512, 512, 263)
evaluating:  0522c0708 (512, 512, 160, 1, 8) (512, 512, 159)


IndexError: boolean index did not match indexed array along dimension 2; dimension is 160 but corresponding boolean dimension is 159

In [76]:
sample_prob = os.path.join(seg_prob_dir, "0522c0598_niftynet_out.nii.gz")
sample_ref = os.path.join(seg_ref_dir, "0522c0598_segmentation.nii.gz")

prob_img = nib.load(sample_prob)
ref_img = nib.load(sample_ref)

prob_np = prob_img.get_fdata()
ref_np = ref_img.get_fdata().astype(np.uint8)

In [18]:
print(prob_img.get_fdata().shape, prob_ref.get_fdata().shape)

(512, 512, 135, 1, 8) (512, 512, 135)


In [87]:
for key, value in defs.LABELS.items():
    label_prob = prob_np[...,value].squeeze()[ref_np == value]
    cnt, minmax, mean, std, _, _ = stats.describe(label_prob)
    median = np.median(label_prob)
    print(key, cnt, minmax, mean, std, median)

BrainStem 5825 (3.96249470213661e-06, 1.0) 0.9044478786417931 0.06826904683904156 0.9999754428863525
Parotid_L 8077 (1.565431873018497e-08, 1.0) 0.8888049556507017 0.08343390463160404 0.9999872446060181
OpticNerve_R 146 (4.503764102992136e-06, 1.0) 0.8595461799008985 0.11582381645177779 1.0
Parotid_R 7846 (3.1416800538863754e-07, 1.0) 0.8879305652714555 0.08220605726737584 0.9998632669448853
Mandible 12879 (8.333314482911192e-09, 1.0) 0.8482818027497093 0.11512122308216441 0.9999936819076538
OpticNerve_L 154 (7.649421718269878e-08, 1.0) 0.8340159570164173 0.13202929717018744 1.0
Chiasm 274 (4.257166779098043e-07, 1.0) 0.3191276427539802 0.20889703824034186 0.00019908894319087267


In [78]:
print("0522c0598_niftynet_out.nii.gz"[:9])

0522c0598


In [ ]:
dispaly(df.groupby(['organ']).mean())